In [ ]:
!pip install -q langchain_openai==0.3.34
!pip install -q birdnet==0.1.7 librosa==0.11.0 soundfile==0.13.1
!pip install -q fastmcp==2.12.4 mcp_use==1.3.11 asyncio==4.0.0 

In [ ]:
from getpass import getpass

api_key = getpass('Enter your API key: ')

In [ ]:
!curl -s -L "https://llm-api.cyverse.ai/v1/models" \
  -H "Authorization: Bearer {api_key}" \
  -H 'Content-Type: application/json' | json_pp

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="js2/llama-4-scout",
    api_key=api_key,
    base_url="https://llm-api.cyverse.ai/v1"
)

In [ ]:
print (llm.invoke("Hello, world!"))

In [ ]:
import subprocess
import time

# Start the server as a background process
server_process = subprocess.Popen(
    ["python", "birdnet_mcp_server.py"],
)

# Wait a moment for the server to start up
time.sleep(5) 
print("MCP Server started in background on port 8000.")

In [ ]:
from mcp_use import MCPAgent
from mcp_use import MCPClient

async def run_the_agent():

    client = MCPClient.from_dict({
        "mcpServers": {
            "birdnet_server": {
                "url": "http://127.0.0.1:8000/mcp"
                # "command": "python", 
                # "args": ["./birdnet_mcp_server.py"],
                # "transport": "stdio" 
            }
        }
    })
    
    # Pass the globally defined 'llm' object
    agent = MCPAgent(llm=llm, client=client, max_steps=5) 
    
    # # --- Execution ---
    user_prompt = "What bird species can be heard in './data/XC589950-gambels-quail.mp3'?"
    print(f"User Prompt: {user_prompt}\n")
    print("--- Agent Execution ---")
    
    # Run the agent and get the final result
    result = await agent.run(user_prompt)
    
    print("\n--- Final Answer ---")
    print(result)

# Execute the asynchronous function using 'await' in Jupyter/IPython
await run_the_agent()